In [2]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [27]:
# Numpy Matrix
# Column : Each song
# Row : Shingles



# Instance variables :
# n : the number of total songs
# s : the number of signatures
# r : the number of signatures in a single band
# b : the number of bands (s // r)
# perm : permutated np array
# mhdata : minhashed (signature array) 
#          row : signatures / columns : songs
# ---------------------------------------------------------------
import numpy as np 
import pandas as pd
import time

class LSH() :
    # Initializer
    # Input
    # data : pandas dataframe for song - shingles
    #        row : songs / column : array of tags for each song
    # tag_dic : dictionary with key : tag / value : integer
    # sig_n : the number of signatures
    # band_r : the number of signatures in a single band
    # bucket_sz : bucket size for LSH
    # -----------------------------------------------------------
    def __init__(self, data, tag_dic, sig_n, band_r) :
        self.data = data
        self.tag_dic = tag_dic
        self.n = data.shape[0]
        self.s = sig_n
        self.r = band_r
        self.bucket_sz = 20000
        if self.s%self.r != 0 :
            print("The number of signatures is not divided by the number of band_r.")
            print("Warning : Remainder signatures will be ignored")
            self.s = self.s - self.s%self.r
        self.b  = self.s // self.r
        self.song_dic = dict(zip(self.data.iloc[:,0],range(len(self.data.iloc[:,0]))))   #index 0 might change
        # converts array of tag to array of integers
        self.data['tags'] = [[tag_dic.get(key) for key in i] for i in data.iloc[:,1]]   #index 1 might change
            
    # permutation
    # save random shuffle of 1:n 
    def permute(self, tag_n) :
        self.perm = np.random.permutation(tag_n)
        
    # This function uses permutation to hash, outputs the minimum value
    # tag_n : Total number of tags (required for permutation)
    # This algorithm could be improved using
    # https://datalab.snu.ac.kr/~ukang/courses/19S-DM/L5-lsh.pdf page 42 later
    def minhash(self, tag_n = None) :   
        # result matrix
        # row : signatures / columns : songs
        s_t = time.time()
        sig_mat = np.zeros((self.s, self.n))
        for sig_num in range(self.s) :
            self.permute(tag_n)
            new_sig = [min(np.take(self.perm, i)) for i in self.data.iloc[:,1]]  # index 1 could be changed later
            sig_mat[sig_num, :] = new_sig
            print("Minhash {0} : {1}".format(sig_num, time.time()-s_t))
        self.mhdata = sig_mat
        
    
    # This function locally hashes the signature vector (of size self.r)
    def local_hash(self, sig_vec) :
        return hash(str(sig_vec)) % self.bucket_sz
    
    # Computes locally sensitive hashing algorithm (put into buckets)
    # Bucket_mtx : row - bands, column - bucket for each band 
    def lsh(self) :
        mh_t = np.transpose(self.mhdata)
        bucket_mtx = np.empty((self.b, self.bucket_sz), object)
        for band_i in range(self.b) :
            s_t = time.time()
            curr = band_i * self.r 
            hashed = [self.local_hash(col[curr:curr+self.r]) for col in mh_t]
            for ind, item in enumerate(hashed, start=0): 
                bucket_mtx[band_i][item] = np.append(bucket_mtx[band_i][item], ind)
            print("Band {0} : {1}".format(band_i, time.time()-s_t))        
        self.bucket_mtx = bucket_mtx
        
    def lsh_dic(self) :
        mh_t = np.transpose(self.mhdata)
        bucket_dic = {}
        for band_i in range(self.b) :
            s_t = time.time()
            curr = band_i * self.r 
            hashed = [self.local_hash(col[curr:curr+self.r]) for col in mh_t]
            for ind, item in enumerate(hashed, start=0): 
                try : 
                    bucket_dic[(band_i,item)] = np.append(bucket_dic[(band_i,item)], ind)
                except KeyError :
                    bucket_dic[(band_i,item)] = [ind]               
            print("Band {0} : {1}".format(band_i, time.time()-s_t))        
        self.bucket_dic = bucket_dic
    
    def mid_level(self, tag_n = None) :
        self.minhash(tag_n)
        self.lsh()
    
    # Outputs the array of similar songs w.r.t given song
    def sim_items(self, song) :
        sim_item = np.array([])
        song_sig = self.mhdata[:,self.song_dic[song]]
        for band_i in range(self.b) :
            s_t = time.time()
            curr = band_i * self.r
            curr_hash_val = self.local_hash(song_sig[curr:curr+self.r])
            bucket_for_this = self.bucket_mtx[band_i][curr_hash_val]
            sim_item = np.union1d(sim_item, list(filter(None, bucket_for_this)))
            print("Find Similar Items / Band {0} : {1}".format(band_i, time.time()-s_t))
        return list(np.take(self.data.iloc[:, 0], sim_item, axis = 0)) # index 0 might change
    
    
        # Outputs the array of similar songs w.r.t given song
    def sim_items_dic(self, song) :
        sim_item = np.array([])
        song_sig = self.mhdata[:,self.song_dic[song]]
        for band_i in range(self.b) :
            s_t = time.time()
            curr = band_i * self.r
            curr_hash_val = self.local_hash(song_sig[curr:curr+self.r])
            bucket_for_this = self.bucket_dic[(band_i, curr_hash_val)]
            sim_item = np.union1d(sim_item, bucket_for_this)
            #print("Find Similar Items / Band {0} : {1}".format(band_i, time.time()-s_t))
        return np.array(np.take(self.data.iloc[:, 0], sim_item, axis = 0)) # index 0 might change

In [28]:
import pandas as pd
data = pd.read_pickle("song_tag_map.pkl")
tag_dict = pd.read_csv("tag_id_dict.csv")
tag_dict.to_dict
tag_dic = tag_dict.set_index('Unnamed: 0')['0'].to_dict()
tag_dic[''] = 28742

In [29]:
#data
# sig_n : the number of signature 
# band_r : the number of bands (for lsh)
# higher band_r requires longer time for each band calculation, but shorter overall running time
# higher band_r will output smaller number of similar items
# band_r should divide sig_n (if possible)
lsh = LSH(data, tag_dic, sig_n = 20, band_r = 5)
lsh.mid_level(28743)
lsh.lsh_dic()
# Should input song number as following form ('10') not 10

Minhash 0 : 2.190100908279419
Minhash 1 : 4.366220951080322
Minhash 2 : 6.59139609336853
Minhash 3 : 8.807657957077026
Minhash 4 : 10.941670894622803
Minhash 5 : 13.088166952133179
Minhash 6 : 15.297792911529541
Minhash 7 : 17.420285940170288
Minhash 8 : 19.79772686958313
Minhash 9 : 22.320438861846924
Minhash 10 : 24.90955901145935
Minhash 11 : 27.099628925323486
Minhash 12 : 29.24092698097229
Minhash 13 : 31.385441064834595
Minhash 14 : 33.468711853027344
Minhash 15 : 35.520264863967896
Minhash 16 : 37.64385199546814
Minhash 17 : 39.73874878883362
Minhash 18 : 41.86536502838135
Minhash 19 : 44.039198875427246
Band 0 : 20.332006692886353
Band 1 : 20.95531702041626
Band 2 : 21.09306287765503
Band 3 : 20.43804907798767
Band 0 : 20.274312019348145
Band 1 : 21.062299966812134
Band 2 : 21.546303272247314
Band 3 : 21.492562770843506


In [76]:
#후보가 되는 곡들의 dictionary를 출력
def answer(songs_know, lsh):
    # ------------ input -------------
    # songs_know : playlist에 있는 songs들을 담은 list
    
    
    candidate_songs = dict()
    
    for song in songs_know:
        
        #함수 이름 앵버가 짜면 바꿔야 함
        if not(str(song) in list(lsh.data['songs'])):
            continue
        toadd = lsh.sim_items_dic(str(song))
        
        # candidate_songs에서 얼만큼 언급되었는지 세어준다
        for it in toadd:
            if(it in candidate_songs):
                candidate_songs[it] += 1
            else: #처음 등장한 녀석
                candidate_songs[it] = 1
    
    candidate_songs = dict(sorted(candidate_songs.items(), reverse=True, key=lambda item: item[1]))
    
    return candidate_songs


#answer2가 진짜 답을 추출함
def answer2(candidate_songs_dict, songs_know):
    real_candidate = list(candidate_songs_dict.keys())
    for song in songs_know:
        if(str(song) in real_candidate):
            real_candidate.remove(str(song))
    if(len(real_candidate) < 100):
        #add_baseline(real_candidate)
        pass
    else:
        real_candidate = real_candidate[0:100]
    
    rc2 = []
    for song in real_candidate:
        rc2.append(int(song))
    
    return rc2


def trythisone(songs_know, lsh):
    answer = []
    
    for song in songs_know:
        if not(str(song) in list(lsh.data['songs'])):
            continue
        toadd = list(lsh.sim_items_dic(str(song)))
        
        answer = list(set(answer + toadd))
        
        if(len(answer) >= 100):
            break
            
    if(len(answer) >= 100):
        return [int(i) for i in answer[0:100]]
    else:
        return list(range(1,101))

In [77]:
valid = pd.read_json('val.json', typ = 'frame')

In [78]:
import json

file_path = "./results2.json"

datum = []

for index, row in valid.iterrows():
    res = trythisone(row['songs'], lsh)
    datum.append({'id' : row['id'] , 'songs' : res, 'tags' : ['타린','간지러워','점수','에드시런셋리스트','첩보영화','우아한','화사한','피아노','힙합스타일','싱숭생숭']})
    if(index % 500 == 0):
        print(index)

with open(file_path, 'w') as outfile:
    json.dump(datum, outfile)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000


In [24]:
results = pd.read_json('results2.json', typ = 'frame')

In [41]:
#results만 제출하려서 빈 곳들(곡이 없는 경우)에 뭐라도 채워 넣어야 해서 채워넣음

file_path = "./secondresults.json"

datum = []

for index, row in results.iterrows():
    if(len(row['songs']) != 100):
        datum.append({'id' : row['id'] , 'songs' : res2, 'tags' : ['타린','간지러워','점수','에드시런셋리스트','첩보영화','우아한','화사한','피아노','힙합스타일','싱숭생숭']})
    else:    
        datum.append({'id' : row['id'] , 'songs' : row['songs'], 'tags' : ['타린','간지러워','점수','에드시런셋리스트','첩보영화','우아한','화사한','피아노','힙합스타일','싱숭생숭']})
    if(index % 500 == 0):
        print(index)

with open(file_path, 'w') as outfile:
    json.dump(datum, outfile)

,id,songs,tags
0,118598,"[642087, 680584, 54195, 375411, 381087, 472456...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
1,131447,[],"[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
2,51464,"[110911, 134401, 224670, 264288, 284009, 34673...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
3,45144,"[160925, 230582, 286975, 367905, 375907, 41986...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
4,79929,"[290421, 464118, 106208, 107570, 14343, 149375...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
5,138538,"[100105, 100127, 100453, 100820, 101283, 10143...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
6,127575,"[14069, 142686, 148498, 174250, 219583, 223363...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
7,115317,"[360566, 156818, 268976, 271064, 282469, 29723...","[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
8,80810,[],"[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."
9,142007,[],"[타린, 간지러워, 점수, 에드시런셋리스트, 첩보영화, 우아한, 화사한, 피아노, ..."


In [75]:
datum

[{'id': 118598,
  'songs': [644079,
   640905,
   590806,
   275346,
   373313,
   563557,
   659521,
   160041,
   127445,
   242159,
   430919,
   251889,
   642713,
   126796,
   127481,
   210628,
   151189,
   24016,
   502007,
   648503,
   680113,
   304527,
   470379,
   228552,
   450241,
   642138,
   438507,
   426967,
   278204,
   665448,
   122193,
   54090,
   532982,
   151080,
   560854,
   676778,
   189617,
   318946,
   523299,
   397882,
   506623,
   191409,
   697770,
   169956,
   629896,
   403593,
   97178,
   421952,
   178058,
   1628,
   165343,
   20271,
   466188,
   323327,
   623454,
   600856,
   540535,
   566506,
   152578,
   554044,
   140718,
   52029,
   131897,
   155328,
   266903,
   129112,
   231994,
   695353,
   372245,
   175171,
   36745,
   310883,
   73007,
   31842,
   12972,
   252915,
   269952,
   356674,
   102749,
   662616,
   272085,
   41163,
   611581,
   108763,
   220665,
   686904,
   528059,
   80571,
   499918,
   131407